In [ ]:
# Импортирование модулей сервиса
from ConversationManager import ConversationManager
from NLU_classifier      import NLU_Classifier
from VicunaBot           import VicunaBot

from typing import List
from langchain.schema import BaseMessage

# Библиотеки машинного обучения и моделирования
from transformers import LlamaForCausalLM, LlamaTokenizer

class PromptGenerator:
    """
    Класс для генерации подсказок для модели LLM.
    """

    def __init__(self, nlu_classifier: NLU_Classifier, vicuna_bot: VicunaBot):
        """
        Инициализация генератора подсказок.

        :param nlu_classifier: объект классификатора NLU для извлечения контекста
        :param vicuna_bot: объект VicunaBot для доступа к глобальному состоянию бота
        """
        self.nlu_classifier = NLU_Classifier('texts (2).docx')
        self.vicuna_bot = VicunaBot(LlamaForCausalLM, LlamaTokenizer, gen_kwargs = None, ConversationManager)

    def get_prompt(self, dialogue_history, messages: List[BaseMessage]) -> str:
        """
        Генерирует подсказку для модели LLM, используя историю диалога и текущий контекст.
        """
        question = messages[-1].content.lower()

        # Получение контекста, связанного с вопросом
        is_relevant, context = self.nlu_classifier.get_context(question)

        # Извлечение именованных сущностей
        named_entities_content = self.nlu_classifier.extract_named_entities(question)

        # Формирование подсказки с использованием контекста и сущностей
        prompt = self._create_prompt(question, context, named_entities_content)

        # Обновить контекст и вопрос в VicunaBot
        self._update_vicuna_bot_state(question, context)

        return prompt

    def _create_prompt(self, question: str, context: str, named_entities_content: str) -> str:
        """
        Создает текст подсказки на основе вопроса, контекста и информации, полученной от извлечения именованных сущностей.

        Этот метод формирует подсказку, которая будет использоваться моделью LLM для генерации ответа. Он интегрирует
        общий контекст вопроса, полученный из NLU_Classifier, и дополнительную информацию, связанную с именованными
        сущностями, для создания более информативной и контекстно-зависимой подсказки.

        Параметры:
            question (str): Вопрос пользователя, который нужно проанализировать.
            context (str): Контекст, связанный с вопросом, полученный из NLU_Classifier.
            named_entities_content (str): Дополнительная информация, связанная с именованными сущностями, полученная из NLU_Classifier.

        Возвращает:
            str: Сформированная подсказка для модели LLM, содержащая вопрос пользователя, контекст и информацию об именованных сущностях.
        """
        if context or named_entities_content:
            prompt = f"USER: {question}\nASSISTANT: {context}\n\n{named_entities_content}"
        else:
            prompt = f"USER: {question}\nASSISTANT: К сожалению, у меня нет информации по этому запросу. Можете уточнить вопрос?"
        return prompt

    def _update_vicuna_bot_state(self, question: str, context: str):
        """
        Обновляет состояние VicunaBot с новым вопросом и контекстом.

        :param question: вопрос пользователя
        :param context: контекст, связанный с вопросом
        """
        self.vicuna_bot.context  = context or None
        self.vicuna_bot.question = question
        self.vicuna_bot.prompt   = self._create_prompt(question, context)